In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.chdir("/content/gdrive/My Drive/4995_Competition/4995_kaggle_competition")

# Data Processing

### Load price data

In [0]:
import pandas as pd
import numpy as np

# fix random seed for reproducibility
np.random.seed(7)

prices = pd.read_csv('train.csv')
dates = prices['Unnamed: 0']
prices = prices.iloc[:,1:].set_index(dates)
prices.index.name = 'date'
prices.head()

###Calculate Returns


In [0]:
rets = (prices.shift(1)-prices)
rets.head()

### Fill NaN values

In [0]:
# Fill prices
prices = prices.fillna(method='bfill', axis='rows', inplace=False)
prices = prices.fillna(0, axis='rows', inplace=False)

# Fill returns
rets = rets.fillna(0, axis='rows', inplace=False)
rets.head()

In [0]:
# Confirm all non null values
for i in range(rets.shape[1]):
    assert not np.isnan(rets.iloc[:, i]).any() or not np.isnan(prices.iloc[:, i]).any()
print("Success")

In [0]:
rets.shape[0]

### Manually create dataset for recurrent network
- default lookback period: 50 days
- sample dimension: lookback period x 505 stocks

#### Create X values

In [0]:
from sklearn.preprocessing import MinMaxScaler

N_STOCKS = 505
WINDOW = 50
BIAS = 0.01

def create_data(prices, window, pred_length=152, normalize=True):
    """
    creates data samples of window length x N_STOCKS using the
    previous window length days
    return: [0-49, 1-50, 2-52, .., 841-890]
    """
    X_fcc = []
    X_lstm = []

    # Scale values between 0 and 1
    if normalize:
        scaler = MinMaxScaler(feature_range=(0, 1))

    for i in range(prices.shape[0]-window):
        labels = prices.iloc[i:i+window,:].to_numpy()
        if normalize:
            labels = scaler.fit_transform(labels)
        
        X_lstm.append(labels)

        # Flatten values for FCC
        X_fcc.append(labels.flatten())
        
    return np.array(X_fcc), np.array(X_lstm)

X_fcc, X_lstm = create_data(prices, WINDOW, normalize=True)
y = create_targets(rets, WINDOW, pred_length=152)

X_fcc.shape, X_lstm.shape, y.shape

### Create Y targets (method 1)

In [0]:
def get_weights(sharpe_vals, i):
    # calc norm values
    pos_norm = 0
    neg_norm = 0
    for val in sharpe_vals:
        if val > 0:
            pos_norm += val
        else:
            neg_norm += val

    # normalize values
    pos_sum = 0
    neg_sum = 0
    for i in range(len(sharpe_vals)):
        if sharpe_vals[i] > 0:
            sharpe_vals[i] = sharpe_vals[i]/pos_norm
            pos_sum += sharpe_vals[i]
        else:
            sharpe_vals[i] = sharpe_vals[i]/(-neg_norm)
            neg_sum += sharpe_vals[i]

    # scale to 1 given bias
    scale_factor = 1-BIAS
    weights = sharpe_vals*scale_factor

    # error checking
    assert weights.idxmax() == sharpe_vals.idxmax()
    assert weights.idxmin() == sharpe_vals.idxmin()
    if np.isnan(weights).any():
        print(sharpe_vals, i)
        raise Exception

    return weights

def validate_values(targets):
    for i in range(len(targets)):
        if np.isinf(targets[i]) or np.isnan(targets[i]) or np.isneginf(targets[i]):
            targets[i] = 0
    assert targets.shape == (N_STOCKS,)
    return targets

def create_targets(rets, window, pred_length=152):
    """
    returns target weights: [50, 51, ..., 739]
    """
    targets = []

    for i in range(window, rets.shape[0]-pred_length):
        # expected returns/std
        ev = rets.iloc[i:i+pred_length, :-1].mean()
        std = rets.iloc[i:i+pred_length, :-1].std()

        # sharpe vals
        sharpe_vals = validate_values(ev / std)

        # get weights
        stock_weights = get_weights(sharpe_vals, i)

        # add to targets
        targets.append(np.hstack([stock_weights, [BIAS]]))  
    return np.array(targets)

### Create Y targets (Method 2): Efficient Frontier

In [0]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

def calc_targets(prices):
    targets = []
    for i in range(152, prices.shape[0]):
        # calc expected returns
        avg_returns = expected_returns.mean_historical_return(prices.iloc[i-152:i,:])

        # calc diagonal covariance matrix
        cov_mat = risk_models.sample_cov(prices.iloc[:150,:])
        diag_mat = cov_mat*np.identity(N_STOCKS)

        # find optimal weights
        ef = EfficientFrontier(avg_returns, cov_mat)
        weights = ef.min_volatility()

        # truncate and round values
        cleaned_weights = ef.clean_weights()
        targets.append(cleaned_weights)
    
    return targets

y_ef = calc_targets(prices)

### Train/Test Split

In [0]:
def split_train_data(X):
    """
    return: X_train, y_train, X_test, y_test
    """
    return X[:689,:], X[689:,:]

X_train_fcc, X_test_fcc = split_train_data(X_fcc)
X_train_lstm, X_test_lstm = split_train_data(X_lstm)

# FCC Neural Net
### Create Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, ELU
from keras.optimizers import Adam

def create_fcc_model(lr=0.000001, loss='mse'):
    optim = Adam(lr=lr)

    model = Sequential()
    model.add(Dense(64, input_dim=25300))
    model.add(ELU())
    model.add(Dense(128))
    model.add(ELU())
    model.add(Dense(256))
    model.add(ELU())
    model.add(Dense(256))
    model.add(ELU())
    model.add(Dense(128))
    model.add(ELU())
    model.add(Dense(N_STOCKS+1, activation='tanh'))
    model.compile(optimizer=optim, loss=loss)
    return model

fcc_model = create_fcc_model()
fcc_model.summary()

### Train

In [0]:
fcc_model.fit(X_train_fcc, y, epochs=1000)

# LSTM
### Create Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Dropout, BatchNormalization
from keras.optimizers import Adam

def create_model(lr=0.01, dropout=0.2, loss='mae'):
    optim = Adam(lr=lr)

    model = Sequential()
    model.add(CuDNNLSTM(units=128, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(CuDNNLSTM(units=128, return_sequences=True,))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(CuDNNLSTM(units=128))
    model.add(Dense(units=N_STOCKS))
    model.compile(optimizer=optim, loss=loss)
    return model

lstm = create_model()
lstm.summary()

### Train

In [0]:
lstm.fit(X_train_lstm, y, epochs=1000, shuffle=False)

# Create submission

In [0]:
DAYS_N = 152

def create_submission(model, Xte, prices, file_name):
    # Predictions
    preds = model.predict(Xte).flatten()
    assert len(preds) == DAYS_N*(N_STOCKS+1)

    # Data Id labels
    dataid = []
    stock_names = prices.columns
    for i in range(DAYS_N):
        for stock in stock_names:
            dataid.append(str(i)+'_'+str(stock))
    
    assert len(dataid) == DAYS_N*(N_STOCKS+1)
    dataid = np.array(dataid)

    # Combine for submission
    df = pd.DataFrame({'Id':dataid, 'Predicted': preds})
    df.to_csv(file_name+'.csv', index=False)

    print("Submission: {}.csv Created".format(file_name))

create_submission(fcc_model, X_test, prices, 'sub_fcc')
create_submission(lstm_model, X_test, prices, 'sub_lstm')